In [19]:
'''
This code is being provided to Scott County in order to run additional trail scorecard analyses with other input data. Please note
that the input data will need to follow certain standards described in the functions.
'''

# create the poi area score

arcpy.env.workspace = r'C:\School\PA 5234\Scott County\Scott County.gdb' # insert path to workspace here
poi_layer_name = 'PlacesofInterest' # insert layer name here

arcpy.AddField_management(poi_layer_name, "points", "LONG") # Add a new field called 'points' of type long
arcpy.CalculateField_management(poi_layer_name, "points", 1, "PYTHON3") # Set all 'points' values to 1

buffered_layer = "PlacesofInterest_Buffer" # name of buffer layer
arcpy.Buffer_analysis(poi_layer_name, buffered_layer, "0.5 Miles") # create 0.5 mile buffer

union_layer = "PlacesofInterest_Buffe_Union" # name of union layer
arcpy.Union_analysis("PlacesofInterest_Buffer #", union_layer) # union layer

dissolved_layer = "PlacesofInterest_Bu_Dissolve" # name of dissolve layer
arcpy.Dissolve_management(union_layer, dissolved_layer, "Shape_Area", [["points", "SUM"]], "SINGLE_PART") # dissolve layer while summing points

where_clause = "SUM_points = 24" # select features where SUM_Points equals 24 for all of the circles that don't overlap with any others
arcpy.management.SelectLayerByAttribute(dissolved_layer, "NEW_SELECTION", where_clause)

arcpy.management.CalculateField(dissolved_layer, "SUM_points", 1, "PYTHON3", "") # calculate field to update SUM_Points to 1 for selected features
arcpy.management.SelectLayerByAttribute(dissolved_layer, "CLEAR_SELECTION") # clear the selection

<Result 'PlacesofInterest_Bu_Dissolve'>

In [20]:
'''This code takes in the four layers to create the final surface, the names of the layers and the reclass scores are needed.'''

public_ownership = 'PublicParcels_MetroCTUs' # public parcel layer
land_use = 'GeneralizedLandUse2020_Clip' # generalized land use layer
pop_density = 'Census2020TigerBlock_Scott' # census block layer
poi_score = dissolved_layer # calling back to layer name from earlier cell

union_layers = f'{public_ownership} #;{land_use} #;{pop_density} #;{poi_score} #' # set the union expressesion string based on layer names
union_layer_name = 'Final_Union' # set the output layer name
arcpy.Union_analysis(union_layers, union_layer_name, "NO_FID") # run the union analysis

<Result 'C:\\School\\PA 5234\\Scott County\\Scott County.gdb\\Final_Union'>

In [21]:
'''This code handles areas that fell outside of the spatial extent of the county boundary set by the generalized land use and the census blocks.'''

arcpy.management.SelectLayerByAttribute(union_layer_name, "NEW_SELECTION", "DESC2020 = ''") # query to remove out of boundary polygons
arcpy.management.DeleteFeatures(union_layer_name)

# add any other filtering queries you would need

<Result 'Final_Union'>

In [22]:
'''This code takes in the reclass values that were determined which is a way of generalizing the actual input data to find the 
score values for each of the four branches of the score. The actual values of the reclass can be edited and in this example the
values were found using a quantile classification.'''

density_classification = { # Density Classification
    1: 0,
    2: 127.86,
    3: 1153.93,
    4: 2067.72,
    5: 14145.72
}

sum_points_classification = { # SUM_points Classification
    1: 3,
    2: 6,
    3: 9,
    4: 12,
    5: 24
}

land_use_classification = { # Land Use Classification
    1: ["Major Highway", "Major Railway", "Airport or Airstrip", "Open Water"],
    2: ["Mixed Use Industrial", "Industrial or Utility", "Extractive"],
    3: ["Agricultural", "Farmstead", "Mixed Use Commercial", "Office", "Single Family Detached"],
    4: ["Seasonal/Vacation", "Single Family Attached", "Manufactured Housing Park", "Golf Course", "Retail and Other Commercial"],
    5: ["Undeveloped", "Park, Recreational, or Preserve", "Institutional", "Mixed Use Residential", "Multifamily"]
}

In [23]:
'''This code adds the fields to the final union layer based on the reclass value and final score.'''

fields_to_add = ["public_owned", "density_score", "poi_score", "land_use_score", "final_score"] # these are the scoring fields that are being added
for field in fields_to_add:
    arcpy.AddField_management(union_layer_name, field, "LONG")

# Define the field names based on your dataset
value_fields = ["CO_NAME", "Density", "SUM_Points", "DESC2020"] # these are the fields that have the values that are referenced in the dictionaries above, if they have different names need to change values

# Update fields based on classification dictionaries
with arcpy.da.UpdateCursor(union_layer_name, value_fields + fields_to_add) as cursor: # go through each row of the table and bring in both lists of fields
    for row in cursor:
        # Set public_owned to 5 for 'scott county' and 0 for others
        if row[0] == 'Scott':
            row[4] = 5
        else:
            row[4] = 0
        
        # Density score, run through the dictionary and if the value from that row is less than that key's value then set the score to the key 
        for key, value in density_classification.items():
            if row[1] <= value:
                row[5] = key
                break
        # SUM_points score,  run through the dictionary and if the value from that row is less than that key's value then set the score to the key
        for key, value in sum_points_classification.items():
            if row[2] <= value:
                row[6] = key
                break
        # Land use score,  run through the dictionary and if the value from that row is in than that key's value list then set the score to the key
        for key, value in land_use_classification.items():
            if row[3] in value:
                row[7] = key
                break
        # Final score
        row[8] = (row[4] + row[5] + row[6] + row[7])  # Adjust the formula for the final score calculation if needed
        cursor.updateRow(row)

In [3]:
'''This code uses a gap line feature to and intersects it with the surface to break each line at each score change on the surface.'''

line_layer = 'Proposed_Alignment' # input line segments layer
union_layer_name = 'Final_Union'

layers_to_intersect = f'{line_layer} #;{union_layer_name} #'
intersect_name = "Proposed_Alignment_Intersect"
arcpy.analysis.Intersect(layers_to_intersect, intersect_name) # intersect (similar to union) the line and surface

<Result 'C:\\School\\PA 5234\\Scott County\\Scott County.gdb\\Proposed_Alignment_Intersect'>

In [5]:
'''This code will use the weighted average of length of the split lines to calculate the average score of each segment.'''

new_field = "scoreXlength" # new field name
expression = "!final_score! * !Shape_Length!" # new field expression

arcpy.management.CalculateField(intersect_name, new_field, expression, "PYTHON3", "", "DOUBLE") # use the field calculator to get score times length

dissolve_layer = "Proposed_Alignment_Scoring" # dissolve layer name
arcpy.management.Dissolve(intersect_name, dissolve_layer, "FID_Proposed_Alignment", "scoreXlength SUM") # dissolve by ID and sum the score time length values

<Result 'C:\\School\\PA 5234\\Scott County\\Scott County.gdb\\Proposed_Alignment_Scoring'>

In [7]:
'''Code to create classification for segment length.'''

line_length_classification = { # SUM_points Classification
    5: 344.979553,
    4: 549.762013,
    3: 875.750044,
    2: 1474.132243,
    1: 4405.65987
}

In [8]:
'''This code will find the average surface score and add the corresponding length score value to get the overlap score of each segment.'''

new_field = "weighted_score" # new field name
expression = "!SUM_scoreXlength! / !Shape_Length!" # new field expression
arcpy.management.CalculateField(dissolve_layer, new_field, expression, "PYTHON3", "", "DOUBLE") # use the field calculator to get weighted score times

arcpy.AddField_management(dissolve_layer, 'length_score', "LONG") # add length score field
arcpy.AddField_management(dissolve_layer, 'overall_score', "DOUBLE") # add overall score field

# Update fields based on classification dictionaries
with arcpy.da.UpdateCursor(dissolve_layer, ["weighted_score", "length_score", "overall_score", "Shape_Length"]) as cursor: # go through each row of the table and bring in both lists of fields
    for row in cursor:
        # length score,  run through the dictionary and if the value from that row is more than that key's value then set the score to the key
        for key, value in line_length_classification.items():
            if row[3] <= value:
                row[1] = key
                break
        # Overall score
        row[2] = (row[0] + row[1])  # Adjust the formula for the final score calculation if needed
        cursor.updateRow(row)